In [ ]:
from spaco_py.SpaCoObject import SPACO
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from momentchi2 import wf, hbe

In [ ]:
sf = np.load(
    "/home/krastega0/SpaCo_py/src/spaco_py/sf_mat.npy", allow_pickle=False
)
neighbor = np.load(
    "/home/krastega0/SpaCo_py/src/spaco_py/A_mat.npy", allow_pickle=False
)

In [ ]:
#Loading the coorddinates 
coords = pd.read_excel('/home/krastega0/SpaCo_py/src/spaco_py/coords_brain.xlsx')

In [ ]:
def plot_spatial_heatmap(coords, values, title="Spatial Heatmap", cmap="viridis", point_size=50):
    """
    Plots a discrete heatmap based on coordinates and corresponding values.
    Args:
        coords (numpy.ndarray): Array of shape (n_samples, 2) containing x and y coordinates.
        values (numpy.ndarray): Array of shape (n_samples,) containing values corresponding to the coordinates.
        title (str): Title of the plot.
        cmap (str): Colormap to use for the heatmap.
        point_size (int): Size of the points in the scatter plot.
    """
    # Ensure inputs are NumPy arrays
    coords = np.array(coords)
    values = np.array(values)
    # Create the scatter plot
    plt.figure(figsize=(8, 6))
    scatter = plt.scatter(coords[:, 0], coords[:, 1], c=values, cmap=cmap, s=point_size, edgecolor="k")
    plt.colorbar(scatter, label="Values")
    plt.title(title)
    plt.xlabel("X Coordinate")
    plt.ylabel("Y Coordinate")
    plt.grid(True)
    plt.savefig(f'{title}.png', dpi=300, bbox_inches='tight')
    plt.show()
# Example usage
# Coordinates (x, y)
# coords = coords
# Values corresponding to the coordinates
# values = spac_patterns_X[:, 0]  # Use the first SpaCo pattern for demonstration
# Plot the heatmap
# plot_spatial_heatmap(coords, values, title="First SpaC",point_size=20)

In [ ]:
testObj = SPACO(neighbormatrix=neighbor, sample_features=sf)

In [ ]:
Pspac, Vkt =testObj.spaco_projection()


## Remarks on Fig 1
The distribution of eigenvalues shows the non-random eigenvalues in blue and the random permuted eigenvalues in orange.
The non-random eigenvalues represent the true structure of the data, while the random permuted eigenvalues serve as a null distribution for comparison.
We use the upper 95% confidence interval of the random permuted eigenvalues to determine the significance of the non-random eigenvalues.
The non-random eigenvalues that exceed this threshold are considered significant and indicate the presence of meaningful patterns in the data.
The plot provides a visual representation of the eigenvalue distributions, allowing us to assess the significance of the non-random eigenvalues in relation to the random permuted eigenvalues.
      
      

## Determing a metric to see if this pattern is actually spatially 
Here we see our first spatial pattern but how can we tell if the patterns that we see are actually spatial or they are just artifacts. We look at the length of the projections as the measure of "Smoothness" for a spatial gene 

In [ ]:
graphLaplacian=testObj._cache["spectral_results"][2]
graphLaplacian

In [ ]:
coeefficients = testObj._cache["sigma_eigh"]
coeefficients

In [ ]:
# generating the theoretical null distribution
# first need to create a vector of random chi_squared variables
coeefficients = testObj._cache["sigma_eigh"]
n = len(testObj._cache["sigma_eigh"])
chi2_random = np.random.chisquare(df=1, size=n)

# then we need to scale them by the eigenvalues
theoretical_null = [coeefficients.T @ np.random.chisquare(df=1, size=n) for i in range(10000)]
len(theoretical_null)

In [ ]:
# generate random normal distributed vector
normal_random = np.random.normal(size=Vkt.shape[0])

# generate the test statistic for this random simulated pattern 
rand_test_statistic = []
for i in range(10000):
    rand_projection = np.random.normal(size=Vkt.shape[0]).T @ graphLaplacian @ Vkt
    len_proj = rand_projection.T @ rand_projection
    rand_test_statistic.append(len_proj)
len(rand_test_statistic)

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(rand_test_statistic, bins=50, color='blue', alpha=0.6, label='Random Test Statistic')
plt.hist(theoretical_null, bins=50, color='orange', alpha=0.6, label='Theoretical Null')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Comparison of Random Test Statistic and Theoretical Null Distributions')
plt.legend()
plt.show()